In [2]:
data_path = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
from Tool_Functions.cleaning_data import * 

from Clustering.new_data_set_all import * 
from Clustering.viualize_datas import * 
from Clustering.new_data_set_all import *

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
df = file_to_dataframe(data_path + "fusion_table_final.csv")

In [4]:
df

,ID_ABONNE,ANCIENNETE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,...,5,6,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,SCORE_FIDELITE
0,51972820001,1015,0,1,4,2,0,1,0,0,...,1,1,0,0,1,1,1,1,"[3, 4, 1]",0.205128
1,52267440701,771,0,0,6,1,0,2,0,0,...,0,2,0,0,1,1,2,1,"[5, 5]",0.384615
2,52241008401,812,0,0,9,0,0,3,0,0,...,2,2,2,4,2,3,2,2,"[12, 11, 3]",0.666667
3,51081669301,1002,0,1,6,0,0,2,0,0,...,1,1,3,0,0,0,0,1,"[6, 3]",0.346154
4,51276418001,986,0,0,4,1,0,1,0,0,...,1,1,1,0,0,1,0,1,"[5, 4]",0.346154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780791,51720983201,779,0,0,0,0,0,0,0,3,...,0,0,0,0,3,0,0,0,"[1, 1, 1]",0.076923
780792,50417615401,991,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,"[1, 1, 1]",0.076923
780793,51367033801,949,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,"[1, 1, 1]",0.076923
780794,50896097901,1022,0,0,0,0,0,0,0,4,...,0,0,2,0,0,0,0,0,"[2, 2]",0.153846


In [4]:
def data_frame_cluster(data, columns, centers_inv, clusters, data_id_abo):
    """
    Create a DataFrame containing clusters, cluster centers, and subscriber IDs.

    Args:
    - data: DataFrame containing the data.
    - columns: List of column names.
    - centers_inv: Inverse cluster centers (denormalized).
    - clusters: Cluster number assigned to each sample.
    - data_id_abo: DataFrame containing subscriber IDs.

    Returns:
    - df_clusters: DataFrame containing clusters, centers, and percentage IDs.
    """
    # Add cluster labels and subscriber IDs to the data
    data['KMEANS'] = clusters
    data['ID_ABONNE'] = data_id_abo['ID_ABONNE']

    # Generate DataFrame with cluster information
    df_clusters = percent_abo_conditions(data, 'KMEANS', 'ID_ABONNE')
    df_clusters = df_clusters.sort_values(by='KMEANS')

    # Round and assign cluster centers to the DataFrame
    centers = np.round(centers_inv, decimals=2)
    for j in range(len(columns)):
        df_clusters[columns[j]] = [centers[i][j] for i in range(len(centers))]

    return df_clusters

In [11]:

def visualize_silhouette_datas_all(filename, columns, data_path_results, change_inf=np.nan, change_nan=5):
    """
    Visualize silhouette scores for different numbers of clusters.

    Args:
    - filename: Name of the file containing the data.
    - columns: List of column names to include in clustering.
    - range_n_clusters: List of numbers of clusters to test.
    - data_path_results: Path to save the results.
    - change_inf: Value to use for replacing infinity.
    - change_nan: Value to use for replacing NaN.

    Returns:
    - silhouette_scores: List of silhouette scores.
    """
    # Load the dataset from file
    df = file_to_dataframe(filename)
    data = df[columns]
    
    # Replace infinite values with specified value
    data.replace([np.inf, -np.inf], change_inf, inplace=True)

    # Normalize the data
    scaler = StandardScaler()
    datas = scaler.fit_transform(data)

    data = pd.DataFrame(datas, columns=data.columns)
    data_id_abo = df[['ID_ABONNE']]

    data.replace(np.nan, change_nan, inplace=True)

    clusterer = KMeans(8, random_state=10)
    clusterer.fit(data)

    centers = clusterer.cluster_centers_
    centers_cluster = np.round(scaler.inverse_transform(centers), decimals=2)
    df_cluster = data_frame_cluster(data, columns, scaler.inverse_transform(centers),
                                             clusterer.labels_, data_id_abo)
    
    save_to_csv_file(df_cluster, data_path_results + "clusterall.csv")

    cluster_labels = clusterer.fit_predict(data)

    df['KMEANS'] = clusterer.labels_
    df = df[['ID_ABONNE', 'KMEANS']]

    save_to_csv_file(df, data_path + "clusters_id_all.csv")

    return df

In [12]:
visualize_silhouette_datas_all(data_path + "fusion_table_final.csv",
                                ['Semaine genéreuse_n_REABOS', 'ODD 15 jours TC_n_REABOS',
                                 'SCORE_FIDELITE', 'ANCIENNETE'], data_path)

/var/folders/md/_fklkl9s1z9gj6bpnwj204y40000gn/T/ipykernel_7312/244633033.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace([np.inf, -np.inf], change_inf, inplace=True)
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,ID_ABONNE,KMEANS
0,51972820001,0
1,52267440701,4
2,52241008401,4
3,51081669301,3
4,51276418001,1
...,...,...
780791,51720983201,6
780792,50417615401,5
780793,51367033801,1
780794,50896097901,3
